# Proyecto Redes Neuronales: Clasificador de géneros musicales

## Integrantes:
Jorge Francisco Cortés López - 314330981

Sandra del Mar Soto Corderi - 3157070267

In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

## Manejo de la data

Escogimos un conjunto de datos

In [37]:
data = 'data/musicdata.csv'
data = pd.read_csv(data)

#Comprobamos que no hay valores vacíos en nuestra data
data.isna().sum().sum()

#Eliminamos los duplicados
music_data = data.drop_duplicates(subset=None, keep='first', inplace=False)

#Eliminamos las columnas que consideramos no necesarias del conjunto de datos
unused_col = ['artist_name', 'track_name', 'track_id', 'popularity', 'duration_ms']
music_data.drop(columns=unused_col).reset_index(drop=True)

#Actualizamos los valores restantes de forma que solo se manejen valores numéricos
mode_dict = {'Major' : 1, 'Minor' : 0}
key_dict = {'C' : 1, 'C#' : 2, 'D' : 3, 'D#' : 4, 'E' : 5, 'F' : 6, 
        'F#' : 7, 'G' : 9, 'G#' : 10, 'A' : 11, 'A#' : 12, 'B' : 12}
music_data['time_signature'] = music_data['time_signature'].apply(lambda x : int(x[0]))
music_data['mode'].replace(mode_dict, inplace=True)
music_data['key'] = music_data['key'].replace(key_dict).astype(int)

#Mostramos los datos ya preprocesados
music_data.head()

music_data = OneHotEncoder().fit_transform(music_data).toarray()
music_data

MemoryError: Unable to allocate 930. GiB for an array with shape (232725, 536294) and data type float64

In [ ]:
#X = music_data.drop(columns=['genre'])
#Y = music_data['genre'] 
# Revolvemos los datos
np.random.shuffle(music_data)
# Tomamos el 60% de los ejemplares para el entrenamiento
num_entrenamiento = int(len(music_data) * .6)
# Datos de entrenamiento.
X_train = music_data[:,num_entrenamiento]
Y_train = music_data[:,num_entrenamiento]
# Datos de validación
X_val = music_data[num_entrenamiento,:]
Y_val = music_data[num_entrenamiento,:]
# Se usa lo restante (20%) para pruebas
X_test = music_data[num_entrenamiento,:]
Y_test = music_data[num_entrenamiento,:]

## Modelo

In [ ]:
class Musica(nn.Module):
    def __init__(self,input_size,hidden,output_size):
        '''
        Define las caracteristicas de una red completamente conectada 
        de tres capas, recibe la cantidad de elementos de entrada, el 
        número de capas ocultas y el número de elementos de salida. 
        Entre cada capa agrega una función de activación logistica.
        '''
        super(Musica,self).__init__()
        #Declaramos las tres capas 
        self.c1 = nn.Linear(input_size, hidden)
        self.c2 = nn.Linear(hidden, hidden)
        self.c3 = nn.Linear(hidden, output_size) 
        # Función de error.
        self.loss_fn = nn.CrossEntropyLoss()
        # Algoritmo de optimización.
        self.optimizer = optim.Adam(self.parameters())
        # Error actual de la red.
        self.loss = 0.0
        
    def forward(self,X):
        '''
        Define una función que de como resultado realizar la propagación
        hacia adelante de los elementos de X en la red definida.
        '''
        #Se activa la capa de entrada
        X = self.c1(X)
        #Se usa relu para activar las capas, ya que funciona como una función de activación unitaria
        X = F.relu(X)
        #Se activa la capa oculta
        X = self.c2(X)
        X = F.relu(X)
        #Se activa la capa de salida
        X = self.c3(X)
        return F.log_softmax(X, dim=1)
        
    
    def back_propagate(self,X,Y):
        '''
        Define una función que realice la propagación hacia atras usando 
        la función de error de entropia cruzada.
        '''
        # Calculamos el error.
        self.loss = self.loss_fn(X,Y.long())
        #Vaciamos los gradientes, hacemos una propagación hacia atrás
        # y actualizamos pesos.
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()
        
        
    def train(self,train_X,optimizer,ciclos=100):
        '''
        Define una función de entrenamiento para la red, la cual utilice
        al conjunto de entrenamiento y el algoritmo de optimización que se 
        obtenga como parametro. Al finalizar los ciclos muestra la gráfica 
        del error.
        '''
        data_error = np.zeros(ciclos)
        for t in range(ciclos):
            # Para cada uno de los lotes.
            for data in train_X:
                X, Y = data
                # Feed forward.
                y_pred = self(X.float())
                # Back propagate.
                self.back_propagate(y_pred, Y)
                # Anexamos el error actual.
                data_error[t] = self.loss
        
        # Graficamos el error.
        if ciclos > 1:
            plt.plot(np.arange(ciclos), data_error)
            plt.xlabel('Iteraciones')
            plt.ylabel('Error')
                
    def confusion(self,dataset,numClases):
        '''
        Muestra la matriz de confusión que presenta los valores actuales de
        la red, respecto al conjunto de datos que se decida usar.
        '''
        # Matriz de confusión inicial con dimensión correspondiente al número de clases.
        cmatrix = np.zeros([numClases,numClases], dtype=int)
        for data in dataset:
            X,Y = data
            #Se obtienen las predicciones de la data
            y_pred = self.forward(X.float())
            _, predicted = torch.max(y_pred.data, 1)
            # Se juntan los datos para obtener predicciones de las clases.
            stacked = torch.stack((predicted ,Y),dim=1)
            # Iteramos sobre las predicciones y las clases.
            for p in stacked:
                tl, pl = p.tolist()
                cmatrix[int(tl), int(pl)] = cmatrix[int(tl), int(pl)] + 1
        # Una vez calculada la matriz se grafica
        music_labels = music_data['genre'].unique() 
        df_cm = pd.DataFrame(cmatrix, index = music_labels, columns = music_labels)
        plt.figure(figsize = (20,15))
        sns.set_theme(style="whitegrid")
        #Por cuestiones de estética y facilidad se usa heatmap
        sns.heatmap(df_cm, annot=True, robust=True, fmt="d", cmap="YlGnBu")
        plt.title("Matriz de confusión sin normalizar")
        plt.ylabel('Predicción')
        plt.xlabel('Actual')
        plt.show()